# Youtube Comment scrapping for 2025 Kicks
This documnet contains all the needed code to scrape the the 10 most commented youtube videos comments to do analysis on.

This code is adapted from a capstone project done in spring 2023 by Sriram Kannan.

The source code can be found __[here](https://github.com/sethpar/Nissan_NNA_comment_scraping/blob/main/Nissan_Segment_List.xlsx)__




## Envoirment Prep

You will need the following libraries to run the following code 

1. googleapiclient
2. pandas
1. plotly

These only need to be installed on your machine once. To install them remove the "#" from the code box below and run the chunk.



In [1]:
# This code chunk installs the necessary libraries to run the code. This only has to be done once.
# To install remove the  '#' from the following 3 lines of code and run the chunk.

#%pip install pandas
#%pip install google-api-python-client
#%pip install plotly

## Data prep 

In order to scrape the needed comments an excel file called 'Nissan_Segment_List.xlsx'. An example of this file can be found __[here]( https://github.com/sriram-k96/nissan-capstone_project/blob/main/EV_VehicleSegments.xlsx)__

In [2]:
import pandas as pd

pd.set_option('display.max_rows', 200)

In [3]:
# Read in our Excel file as the Dataframe "segmentdf"
segmentdf = pd.read_excel('Nissan_Segment_List.xlsx', header = 1)
# look at the first few rows to ensure dataframe was read correctly
segmentdf.head()

,Segments,Make Model
0,* ENTRY SUV,NISSAN KICKS


In [4]:
# Remove asterisks, strip whitespace, and capitalize each word in the 'Segments' column
segmentdf['Segments'] = segmentdf['Segments'].apply(lambda x: x.replace('*', '').strip().title())

# Replace 'Suv' with 'SUV' in the 'Segments' column
segmentdf['Segments'] = segmentdf['Segments'].apply(lambda x: x.replace('Suv', 'SUV'))

# Extract the first word from 'Make Model' column and assign it to a new 'Make' column
segmentdf['Make'] = segmentdf['Make Model'].str.split().str.get(0)

# Remove the first word from 'Make Model' and keep the rest as the new 'Make Model'
segmentdf['Make Model'] = segmentdf['Make Model'].str.split().str.slice(1).str.join(' ')

# Rename columns: 'Make Model' to 'Model' and 'Segments' to 'Segment'
segmentdf = segmentdf.rename(columns={'Make Model': 'Model', 'Segments': 'Segment'})

# Reorder columns to 'Segment', 'Make', and 'Model'
segmentdf = segmentdf[['Segment', 'Make', 'Model']]

# Capitalize the first letter of each word in the 'Make' column
segmentdf['Make'] = segmentdf['Make'].apply(lambda x: x.capitalize())

# Capitalize the first letter of each word in the 'Model' column
segmentdf['Model'] = segmentdf['Model'].apply(lambda x: x.capitalize())


In [5]:
# Count the number of vechicles in each segment
segmentdf['Segment'].value_counts()

Segment
Entry SUV    1
Name: count, dtype: int64

In [6]:
# Output segmentdf as an excel file
segmentdf.to_excel('VehicleSegments.xlsx')

In [7]:
# Read in VehicleSegments.xlsx to ensure it reads correctly
df = pd.read_excel('VehicleSegments.xlsx')
# Drop unused row
df.drop('Unnamed: 0', axis = 1, inplace = True)

In [8]:
# Create a plot that shows the segement and make model of one random row
import plotly.graph_objects as go
tab4 = df.sample(1)
fig = go.Figure(data=[go.Table(
    header=dict(values=list(('Make', 'Model')),
                #fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[tab4.Make, tab4.Model],
               #fill_color='lavender',
               align='left'))
])

fig.show()

## Scrape Youtube Comments
This section of this notebook gives the code of how to scrape comments from youtube.
This is done by finding the 10 most commented video for each year, make and model. Then scraping all comments and replies from said videos.

This first chunk of code is the set-up. 
We first load in googleapiclient which let's us find and scrape videos.
We then set the API key to the API we want to us.(More about this later)
Then we build a Youtube service object(this lets)


In [ ]:
from googleapiclient.discovery import build

pd.set_option('display.max_rows', 500)
# Assign your YouTube API key to the variable 'api_key'
api_key = 'Put_key_Here' 
# Build a YouTube service object using the API key
youtube = build('youtube', 'v3', developerKey=api_key)

In [ ]:
# Create a list containing all model years, if you want more then one it should look like ['2023','2024','2025']
year_list = ['2025']

# Read the Excel file 'VehicleSegments.xlsx' into a DataFrame
segmentdf = pd.read_excel('VehicleSegments.xlsx')

# Drop the column named 'Unnamed: 0' from the DataFrame
segmentdf.drop('Unnamed: 0', axis=1, inplace=True)

# Display the DataFrame
segmentdf

In [ ]:
#Remove EVs to Optimize Youtube Queries
#If you need to find EVs replace first line with "seg2 = segmentdf"
#Remove duplicated Grand Cherokee from Midsize SUV
seg2 = segmentdf.drop(segmentdf[(segmentdf['Model'] == 'Ariya') | (segmentdf['Model'] == 'Leaf') | (segmentdf['Model'] == 'C-hr') | (segmentdf['Model'] == 'Bz4x') | (segmentdf['Model'] == 'Bolt') | (segmentdf['Model'] == 'Ioniq') | (segmentdf['Model'] == 'Ioniq 5') | (segmentdf['Model'] == 'Ev6') | (segmentdf['Model'] == 'Nexo') | (segmentdf['Model'] == 'Ram') | (segmentdf['Model'] == 'R1t launch') | (segmentdf['Model'] == 'Endurance') | (segmentdf['Model'] == 'Solterra') | (segmentdf['Model'] == 'Mustang mach-e') | (segmentdf['Model'] == 'Hummer') | (segmentdf['Model'] == 'Id4')].index)
seg2 = seg2.reset_index()
seg2.drop('index', axis = 1, inplace = True)

Count the number of models each make has.

In [ ]:
seg2["Make"].value_counts()

The following code only applies if you want to compare many makes and models. You can replace "Ford" and "Honda" with any make the splits your list in half

In [ ]:
#Youtube API per account cannot handle all models to be scraped at once. So they need to be segregated as shown below. 
#Safest way is to do it one at a time and then switch api key to different accounts.
#Use below code to systematically generate files as per make and then merge them in the end. 
#segloc = seg2.loc[(seg2['Make'] == "Ford") | (seg2['Make'] == "Honda")]

In [ ]:
# Initialize an empty list to store queries
query_list = []

# Iterate over each year in year_list
for i in year_list:
    # Iterate over each model in the 'Model' column of seg2 DataFrame
    for j in seg2["Model"]:
        # Construct a query string with the format 'Make Model Year'
        query = seg2.loc[seg2["Model"] == j]["Make"].tolist()[0] + ' ' + j + ' ' + str(i)
        # Append the constructed query to the query_list
        query_list.append(query)

Show all vechicles queried

In [ ]:
query_list

Can be used to get the list of URLs to be scraped - Choose ones with maximum comments to maintain efficiency of queries. 

But this process is incredibly intensive on the quota and has to be used sparingly. 

Hence a combination of this and manual process of searching and grabbing URLs directly from youtube were used. 

Scaling this requires permission from Google which can be done by the organization if required through increasing Quota limits substantially.

10000 points per day is inadequate. 

Only 2025 will be considered here based on our year list

In [ ]:
# Initialize an empty list to store URLs and video details
urls = []

# Iterate over each query in query_list
for i in query_list:
    # Perform a YouTube search with the query and get the response
    search_response = youtube.search().list(q=i, part='id,snippet', maxResults=10).execute()
    
    # Extract video IDs from the search response
    video_ids = [search_result['id']['videoId'] for search_result in search_response.get('items', [])]
    
    # Get detailed information about the videos using the video IDs
    video_response = youtube.videos().list(id=','.join(video_ids), part='id,snippet,statistics').execute()
    
    try:
        # Iterate over each video result in the video response
        for video_result in video_response.get('items', []):
            # Extract the video title
            title = video_result['snippet']['title']
            
            # Construct the video URL
            url = f'https://www.youtube.com/watch?v={video_result["id"]}'
            
            # Extract the video ID
            video_id = video_result["id"]
            
            # Extract the view count
            view_count = video_result['statistics']['viewCount']
            
            # Extract the comment count
            comment_count = video_result['statistics']['commentCount']
            
            # Append the query, title, URL, video ID, view count, and comment count to the urls list
            urls.append((i, title, url, video_id, view_count, comment_count))
    except:
        # Print the query if an exception occurs
        print(i)
        continue

# Create a DataFrame from the urls list with specified column names
yturl_df = pd.DataFrame(urls, columns=['Query', 'Title', 'URL', 'VideoID', 'ViewCount', 'Comment_Count'])

Now lets find the videos with the most comments for each vehicle.

In [ ]:
yturl_df['Comment_Count'] = yturl_df['Comment_Count'].astype(int)
#Taking only the top result by comments. Due to a large number of vehicles over 2 years, this needs to be done to stay within query limits
yturl_df_top = yturl_df.groupby('Query').apply(lambda x: x.nlargest(10, 'Comment_Count')).reset_index(drop=True)
#Removing duplicate results - Sometimes same results come up as top results for similar queries - Unpopular vehicles for both year queries for instance
#Needs to be done to avoid duplicating scraping. 
yturl_df_top = yturl_df_top.drop(yturl_df_top[yturl_df_top.drop('Query', axis = 1).duplicated()].index)

Now let look at the videos

In [ ]:
yturl_df_top

Now for each video we are going to create a list of its comments and replies.

In [ ]:
# Initialize empty lists to store replies and comments
replies = []
comments = []

# Iterate over each video ID in the 'VideoID' column of yturl_df_top DataFrame
for vid in yturl_df_top['VideoID']:
    # Get the comment threads for the video
    video_response = youtube.commentThreads().list(part='snippet,replies', videoId=vid).execute()
    
    # Get the corresponding model for the video ID
    model = yturl_df_top.loc[yturl_df_top["VideoID"] == vid]["Query"].tolist()[0]
    
    # Construct the video URL
    url = 'https://www.youtube.com/watch?v=' + vid
    
    # Get the video title
    title = yturl_df_top.loc[yturl_df_top["VideoID"] == vid]["Title"].tolist()[0]
    
    while video_response:
        # Iterate over each item in the video response
        for item in video_response['items']:
            # Extract the top-level comment
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            
            # Get the total number of replies to the comment
            replycount = item['snippet']['totalReplyCount']
            
            try:
                # If there are replies, extract each reply
                if replycount > 0:
                    for reply in item['replies']['comments']:
                        # Extract the reply text
                        reply = reply['snippet']['textDisplay']
                        replies.append(reply)
            except:
                # Print an error message if there is an issue with extracting replies
                print('Reply Err')
 
            # If there are no replies, set replies to "None"
            if replies == list():
                replies = "None"
            
            # Append the model, comment, replies, title, and URL to the comments list
            comments.append((model, comment, replies, title, url))
            
            # Reset the replies list for the next comment
            replies = []
 
        # Check if there is a next page of comments
        if 'nextPageToken' in video_response:
            # Get the next page of comments
            video_response = youtube.commentThreads().list(part='snippet,replies', videoId=vid, 
                                                           pageToken=video_response['nextPageToken']).execute()
        else:
            # Break the loop if there are no more pages
            break

Now make a datafrane with the comments and make it a CSV

In [ ]:
df = pd.DataFrame(comments, columns = ['Model', 'Comments', 'Replies', 'Video Title', 'URL'])
df.to_csv('YoutubeComments.csv')

Below is a way to concatinate all comments if each CSV was created for each manufaturer.

In [ ]:
#These files were generated one at a time. 
#df1 = pd.read_csv('YoutubeCommentsHyundai_Kia.csv')
#df2 = pd.read_csv('YoutubeCommentsFord_Honda.csv')
#df3 = pd.read_csv('YoutubeCommentsToyota.csv')
#df4 = pd.read_csv('YoutubeCommentsNissan.csv')
#df5 = pd.read_csv('YoutubeCommentsBuick_Porsche_Gmc.csv')
#df6 = pd.read_csv('YoutubeCommentsVolkswagen.csv')
#df7 = pd.read_csv('YoutubeCommentsChevrolet.csv')
#df8 = pd.read_csv('YoutubeCommentsSubaru_Mazda.csv')
#df9 = pd.read_csv('YoutubeCommentsLast6Make.csv')
#df10 = pd.read_csv('YoutubeCommentsJeep.csv')
#df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]).reset_index()
#df.drop(['index', 'Unnamed: 0'], axis = 1, inplace = True)
#df.to_csv('YoutubeComments.csv')

Now combine the Youtube and Vechicle segment csvs

In [ ]:
# Read the CSV file 'YoutubeComments.csv' into a DataFrame
df = pd.read_csv('YoutubeComments.csv')

# Drop the column named 'Unnamed: 0' from the DataFrame
df.drop('Unnamed: 0', axis=1, inplace=True)

# Extract the first word from the 'Model' column and assign it to a new 'Make' column
df['Make'] = df['Model'].str.split().str.get(0)

# Extract the third word from the 'Model' column and assign it to a new 'Year' column
df['Year'] = df['Model'].str.split().str.get(2)

# Extract the second word from the 'Model' column and keep it as the new 'Model' column
df['Model'] = df['Model'].str.split().str.get(1)

# Read the Excel file 'VehicleSegments.xlsx' into a DataFrame
df2 = pd.read_excel('VehicleSegments.xlsx')

# Drop the column named 'Unnamed: 0' from the DataFrame
df2.drop('Unnamed: 0', axis=1, inplace=True)

# Merge the two DataFrames on the 'Make' and 'Model' columns
df_full = pd.merge(df, df2, on=["Make", "Model"])

# Select and reorder the columns in the merged DataFrame
df_full = df_full[['Segment', 'Make', 'Model', 'Year', 'Comments', 'Replies', 'Video Title', 'URL']]

# Calculate the word count of each comment and assign it to a new 'Word Count' column
df_full['Word Count'] = df_full['Comments'].str.split(' ').str.len()

# Save as a csv
df_full.to_csv('YoutubeCommentsFull.csv')

In [ ]:
# Head dataframe to ensure it looks correct
df_full.head()

In [ ]:
# Reset the index of the DataFrame and assign it to df_final
df_final = df_full.reset_index()

# Drop the 'index' column from the DataFrame
df_final.drop('index', axis=1, inplace=True)

# Remove duplicate rows from the DataFrame
df_final = df_final.drop_duplicates()

# Save the DataFrame to a CSV file named 'YoutubeComments_40_InfoRich.csv'
df_final.to_csv('YoutubeComments_40_InfoRich.csv')

In [ ]:
# Count the occurrences of each unique value in the 'Make' column and display the results
df_final['Make'].value_counts()

In [ ]:
#DEBUG
#Check if all replies which are list of lists + comments add up to total comments in the YT Video.
#[len(i) for i in list(df.loc[df["Replies"] != 'None']['Replies'])]

## Setting up google API key

When this documnet is first released there is an API Key created by Seth Young that will be avaible. If multple people want to use this code at the same time OR the key expires. The following is the steps needed to create a new google API key.

1. Go to https://cloud.google.com/.
1. Click Start Free in top right corner.
1. Create a new google account.
1. After your new account is made and set-up Click console is top right corner.
1. In the top middle search bar, search "YouTube Data API v3"
1. click "Manage"
1. Then click "Credentials"
1. Then on the middle left click "Create Credentials" then "API key"
1. You will then be shown a pop-up with your API key click "copy" then done
1. After than you can paste the API key in the code above.